In [1]:
import geopandas as gpd
import psycopg2
import folium
import folium.plugins
import fiona
import osmnx as ox
import networkx as nx
import operator
import json


# geojson boundary file export from https://wambachers-osm.website/boundaries/

In [2]:
aois = gpd.read_file("../../data/Rapperswil-Jona_AL8.GeoJson")
aois = aois.to_crs({'init': 'epsg:4326'})

# sql query to be used with the planetosm data 

In [8]:
boundary_query = """
SELECT *
FROM ST_Transform(ST_SetSRid(ST_GeomFromGeoJSON('{}'), 4326), 3857) AS geometry
""".format(json.dumps(aois.loc[0]['geometry'].__geo_interface__))


# Graphical Representation

In [9]:
with psycopg2.connect("") as conn:
    boundary = gpd.read_postgis(boundary_query, conn, geom_col='geometry')
    boundary.crs = fiona.crs.from_epsg(3857)
# coordinates for the boundary
default_view = aois.loc[0]['geometry'].representative_point().__geo_interface__['coordinates'][::-1]
m = folium.Map(location=default_view, zoom_start=13, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)
folium.GeoJson(boundary).add_to(m)

m